In [2]:
import torch
from torch import nn
from d2l import torch as d2l


In [3]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [4]:
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

In [5]:
def init_weight(m):
    if type(m) == nn.Linear:
        nn.init.normal(m.weight, std=0.01)

net.apply(init_weight)

/var/folders/xy/ypttxzt12rqg42s3mgf8b2yc0000gn/T/ipykernel_68215/3391898610.py:3: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(m.weight, std=0.01)


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
)

In [6]:
loss = nn.CrossEntropyLoss(reduction='none')

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

In [9]:
class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [11]:
def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[0] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [12]:
def evaluate_accuracy(net, data_iter):  #@save
    """计算在指定数据集上模型的精度"""
    metric = Accumulator(2)  # 正确预测数、预测总数
    for X, y in data_iter:
        metric.add(accuracy(net(X), y), d2l.size(y))
    return metric[0] / metric[1]